In [ ]:
#Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
import os
import cv2
from random import randint
from IPython.display import SVG
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
import PIL
from PIL import Image

import keras

In [ ]:
#Mount Google Drive in Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing the datasets

train_dir = "/content/drive/MyDrive/FLAME Dataset/Training.zip (Unzipped Files)"
test_dir = "/content/drive/MyDrive/FLAME Dataset/Test.zip (Unzipped Files)"

# Classifying the two classes of datasets

In [ ]:
#Count of training images

cat_counts = {}
for cat in os.listdir(train_dir):
    counts = len(os.listdir(os.path.join(train_dir, cat)))
    cat_counts[cat] =counts
print(cat_counts)

{'Training': 2}


In [ ]:
#count of test images

cat_counts = {}
for cat in os.listdir(test_dir):
    counts = len(os.listdir(os.path.join(test_dir, cat)))
    cat_counts[cat] =counts
print(cat_counts)

{'Test': 2}


# Data pre-processing 

In [ ]:
#Data Preprocessing
#Image Data Generator

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,   
                                   rotation_range = 40, 
                                   zoom_range = 0.2,    
                                   brightness_range = [0.8, 1.2], 
                                   horizontal_flip = True,     
                                   fill_mode = 'nearest',
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2
                                   ) 

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/FLAME Dataset/Training.zip (Unzipped Files)',
                                                  target_size = (224, 224),
                                                  batch_size = 32,
                                                  class_mode = 'binary', shuffle = True)

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/FLAME Dataset/Test.zip (Unzipped Files)',
                                             target_size = (224, 224),
                                             batch_size = 32,
                                             class_mode = 'binary')

Found 39375 images belonging to 1 classes.
Found 8617 images belonging to 1 classes.


In [ ]:
#Finding the class Index
training_set.class_indices

{'Training': 0}

# Balancing the data

In [ ]:
#Using class_weights to balance the training data

from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(training_set.classes), 
                training_set.classes) 

class_weights = dict(enumerate(class_weights))

In [ ]:
#Printing the evaluated class_weights

class_weights

{0: 1.0}

In [ ]:
#Importing required additional libraries

from keras.layers import Dropout
from keras import models, regularizers, layers, optimizers, losses, metrics

from keras.layers import GlobalAveragePooling2D
from keras.layers import GaussianNoise

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization
from keras.utils import np_utils

# Finding optimal model. In this instance it is VGG19

In [ ]:
#Importing libraries for VGG19

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

vgg_base = VGG19(weights='imagenet',    
                 include_top=False,     
                 input_shape=(224, 224, 3))
print(vgg_base.summary())

80150528/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

# Fine tuning 

In [ ]:
#Fine tuning the model

trainable = False

for layer in vgg_base.layers:
    if layer.name == 'block5_conv2':
        trainable = True
        
    layer.trainable = trainable
    
print(vgg_base.summary())

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
#Model building

def build_model():
    from tensorflow.keras.optimizers import Adam, RMSprop
           
    model = keras.models.Sequential([vgg_base,
                                     keras.layers.Flatten(),
                                     keras.layers.Dropout(0.50),
                                     keras.layers.Dense(1, activation='sigmoid')])
    
    
    # Compiling the model

    model.compile(optimizer=RMSprop(lr = 1e-4), 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model = build_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 25089     
Total params: 20,049,473
Trainable params: 7,104,513
Non-trainable params: 12,944,960
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
#Calculating steps per epoch

BATCH_SIZE = 32
train_steps = training_set.n // BATCH_SIZE
test_steps = test_set.n // BATCH_SIZE

train_steps, test_steps

(1230, 269)

In [ ]:
#Adding 'Early Stopping' 

callback = keras.callbacks.EarlyStopping(
    monitor = "accuracy",
    min_delta = 0,
    patience = 3,
    verbose = 1,
    mode = "auto",
    baseline = None,
    restore_best_weights = False
)

In [ ]:
#Training the model

history = model.fit_generator(
    training_set,
    steps_per_epoch=train_steps, class_weight = class_weights,
    epochs=100, callbacks = [callback]
    )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
24/24 [==============================] - 616s 25s/step - loss: 0.6561 - accuracy: 0.7001
Epoch 2/100
24/24 [==============================] - 606s 25s/step - loss: 0.3374 - accuracy: 0.8487
Epoch 3/100
24/24 [==============================] - 616s 26s/step - loss: 0.2752 - accuracy: 0.8684
Epoch 4/100
24/24 [==============================] - 622s 26s/step - loss: 0.2610 - accuracy: 0.8837
Epoch 5/100
24/24 [==============================] - 624s 26s/step - loss: 0.2634 - accuracy: 0.8941
Epoch 6/100
24/24 [==============================] - 626s 26s/step - loss: 0.2653 - accuracy: 0.9171
Epoch 7/100
24/24 [==============================] - 639s 27s/step - loss: 0.2917 - accuracy: 0.8683
Epoch 8/100
24/24 [==============================] - 640s 27s/step - loss: 0.1674 - accuracy: 0.9456
Epoch 9/100
24/24 [==============================] - 633s 26s/step - loss: 0.1822 - accuracy: 0.9429
Epoch 10/100
24/24 [==============================] - 635s 26s/step - loss: 0.1658 - accura

In [ ]:
#Saving the model

model.save('vgg19_model_19.h5')

In [ ]:
#Loading saved model

from keras.models import load_model
vggfinal = load_model('/content/drive/MyDrive/Wildfire/vgg_model_19.h5')

# Testing the model using a sample test data

In [ ]:
#Testing the model with a sample image of 'FIRE'

import numpy as np
from keras.preprocessing import image

test_image = image.load_img('/content/drive/MyDrive/FLAME Dataset/Test.zip (Unzipped Files)/Test/Fire/resized_test_fire_frame1008.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image / 255.0
result = model.predict(test_image)
training_set.class_indices
if result < 0.5:
    print('Fire')
else:
    print('No Fire')

In [ ]:
#Testing the model with a sample image of 'NO FIRE'

import numpy as np
from keras.preprocessing import image

test_image = image.load_img('/content/drive/MyDrive/FLAME Dataset/Test.zip (Unzipped Files)/Test/No_Fire/resized_test_nofire_frame1012.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image / 255.0
result = model.predict(test_image)
training_set.class_indices
if result < 0.5:
    print('Fire')
else:
    print('No Fire')

Fire


In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []
f1scores = []
iteration = 1

for train_index, test_index in kfold.split(training_set,test_set):

    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=15, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

  #separate train data and test data
    x_train,x_test=training_set[train_index],training_set[test_index]
    y_train,y_test=test_set[train_index],test_set[test_index]

    model.fit(x_train, y_train, epochs=30, batch_size=256,verbose=0)
    scores = model.evaluate(x_test, y_test, verbose=0)
    cvscores.append(scores[1] * 100)
    #calculate F1 score
    y_pred = model.predict(x_test, batch_size=256, verbose=1)
    y_pred = np.where(y_pred > 0.5, 1, 0)
    f1 = f1_score(y_test, y_pred, average='macro')
    f1scores.append(f1)
    iteration = iteration + 1
    
print("Accuracy: %.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
print("F1: %.2f%% (+/- %.2f%%)" % (numpy.mean(f1scores), numpy.std(f1scores)))